In [1]:


import folium
from folium.plugins import HeatMap
import pandas as pd
import geopandas as gpd
import numpy as np
import math


In [2]:
df_full_tdrive = pd.read_csv(r'C:\Users\ss6365\Desktop\location_privacy_final\tdrive\data\merged_all_security_subset_3km.csv')
df_full_collected = pd.read_csv(r'C:\Users\ss6365\Desktop\Datasets\Collected\filtered_data.csv')
df_full_geolife = pd.read_csv(r'C:\Users\ss6365\Desktop\location_privacy_final\geolife\data\merged_all_security_subset_3km.csv')



In [3]:
fraction_of_rows = 0.2


df_tdrive = df_full_tdrive.sample(frac=fraction_of_rows, random_state=1) 
df_collected = df_full_collected.sample(frac=fraction_of_rows, random_state=1) 
df_geolife = df_full_geolife.sample(frac=fraction_of_rows, random_state=1) 


your_dataframe = df_full_geolife
# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(
    your_dataframe,
    geometry=gpd.points_from_xy(your_dataframe.longitude, your_dataframe.latitude))


In [14]:



your_dataframe = df_full_geolife
# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(
    your_dataframe,
    geometry=gpd.points_from_xy(your_dataframe.longitude, your_dataframe.latitude))

# Determine the bounds of your data
bounds = gdf.total_bounds  # returns (minx, miny, maxx, maxy)
map_center = [(bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2]

# Your Mapbox access token
mapbox_access_token = 'sk.eyJ1Ijoic2hhZml6dXJyYWhtYW5zZWVhbSIsImEiOiJjbHUwbGt0ZGcwNHI4MmludmNza252aHlzIn0.eQWfr353P4zFLKHiytBOnw'

# Use a predefined Mapbox style
mapbox_style = 'light-v11'  # Example style

# Create the map with the Mapbox style
m = folium.Map(
    location=map_center,
    zoom_start=13,
    tiles='https://api.mapbox.com/styles/v1/mapbox/' + mapbox_style + '/tiles/{z}/{x}/{y}?access_token=' + mapbox_access_token,
    attr='Mapbox'
)

# Create a heatmap
heatmap_data = gdf[['latitude', 'longitude']].values.tolist()
heatmap = HeatMap(heatmap_data, min_opacity=.5, radius=6, blur=20, max_zoom=1)
m.add_child(heatmap)

# Add a rectangle to represent the bounds
folium.Rectangle(
    bounds=[[bounds[1], bounds[0]], [bounds[3], bounds[2]]],
    color=' #000000',
    fill_opacity=1,
    #fill = True,
     weight=2,
).add_to(m)

# Define an outer box with an offset
offset = 0.01  # Adjust as necessary for the desired spacing
outer_bounds = [[bounds[1] - offset, bounds[0] - offset], [bounds[3] + offset, bounds[2] + offset]]

# Create the dotted outer box
folium.PolyLine(
    locations=[outer_bounds[0], [outer_bounds[0][0], outer_bounds[1][1]], outer_bounds[1], [outer_bounds[1][0], outer_bounds[0][1]], outer_bounds[0]],
    color='  red',

    weight=2,
).add_to(m)

# Display the map in the Jupyter Notebook


# Function to adjust longitude size to maintain a square appearance
def adjust_longitude_size(lat, size_lat):
    R = 6378.1  # Earth's radius in kilometers
    lat_rad = math.radians(lat)  # Convert latitude to radians
    size_lon_km = R * math.cos(lat_rad) * math.pi / 180  # Size of 1 degree of longitude in km at the given latitude
    size_lat_km = 111.32  # Approximate size of 1 degree of latitude in km (constant)
    size_lon = size_lat * (size_lat_km / size_lon_km)  # Adjust size in degrees of longitude
    return size_lon

# Assuming 'bounds' contains your GeoDataFrame's total bounds
num_squares_lat = 10 # Number of squares along the latitude
lat_range = np.linspace(bounds[1], bounds[3], num_squares_lat + 1)  # Range of latitudes for the squares

for i in range(num_squares_lat):
    # Average latitude of the current square
    avg_lat = (lat_range[i] + lat_range[i+1]) / 2
    # Size of each square in degrees of latitude
    size_lat = lat_range[i+1] - lat_range[i]
    # Adjusted size of each square in degrees of longitude
    size_lon = adjust_longitude_size(avg_lat, size_lat)
    
    for j in range(num_squares_lat):
        # Longitude range for the squares, adjusted for each latitude
        lon_min = bounds[0] + j * size_lon
        lon_max = bounds[0] + (j + 1) * size_lon
        # Defining the bounds of the current square
        square_bounds = [[lat_range[i], lon_min], [lat_range[i+1], lon_max]]
        
        # Drawing the square on the map
        folium.Rectangle(
            bounds=square_bounds,
            color=' #000000',
            fill=False,
           
            fill_opacity=0.1,
            weight=.6,
        ).add_to(m)

# Display the map
m

# map_html = 'C:\\Users\\ss6365\\Desktop\\VisualCodeImplementation\\main.html'
# m.save(map_html)

In [6]:
import folium
import numpy as np
import math
import geopandas as gpd

# Function to adjust longitude size to maintain a square appearance
def adjust_longitude_size(lat, size_lat):
    R = 6378.1  # Earth's radius in kilometers
    lat_rad = math.radians(lat)  # Convert latitude to radians
    size_lon_km = R * math.cos(lat_rad) * math.pi / 180  # Size of 1 degree of longitude in km at the given latitude
    size_lat_km = 111.32  # Approximate size of 1 degree of latitude in km (constant)
    size_lon = size_lat * (size_lat_km / size_lon_km)  # Adjust size in degrees of longitude
    return size_lon

# Assuming 'bounds' contains your GeoDataFrame's total bounds
num_squares_lat = 10  # Number of squares along the latitude
lat_range = np.linspace(bounds[1], bounds[3], num_squares_lat + 1)  # Range of latitudes for the squares

for i in range(num_squares_lat):
    # Average latitude of the current square
    avg_lat = (lat_range[i] + lat_range[i+1]) / 2
    # Size of each square in degrees of latitude
    size_lat = lat_range[i+1] - lat_range[i]
    # Adjusted size of each square in degrees of longitude
    size_lon = adjust_longitude_size(avg_lat, size_lat)
    
    for j in range(num_squares_lat):
        # Longitude range for the squares, adjusted for each latitude
        lon_min = bounds[0] + j * size_lon
        lon_max = bounds[0] + (j + 1) * size_lon
        # Defining the bounds of the current square
        square_bounds = [[lat_range[i], lon_min], [lat_range[i+1], lon_max]]
        
        # Drawing the square on the map
        folium.Rectangle(
            bounds=square_bounds,
            color='blue',
            fill=False,
           
            fill_opacity=0.2,
            weight=.3,
        ).add_to(m)

# Display the map
m


NameError: name 'bounds' is not defined

In [7]:
import folium
from folium import Map, Rectangle
import geopandas as gpd
import numpy as np

# Assuming your GeoDataFrame 'gdf' and Folium map 'm' are already defined as before

# Determine the bounds of your data
bounds = gdf.total_bounds  # returns (minx, miny, maxx, maxy)

# Calculate the size of each grid cell
num_cells = 10  # Use a 10x10 grid instead of 40x40 for simplicity
lat_range = np.linspace(bounds[1], bounds[3], num_cells + 1)
lon_range = np.linspace(bounds[0], bounds[2], num_cells + 1)

# Iterate over the grid
for lat in range(num_cells):
    for lon in range(num_cells):
        # Define the bounds of the rectangle for the current grid cell
        bottom_left = [lat_range[lat], lon_range[lon]]
        top_right = [lat_range[lat + 1], lon_range[lon + 1]]
        
        # Draw the rectangle (grid cell)
        folium.Rectangle(
            bounds=[bottom_left, top_right],
            color='#000000',  # Grid line color
            fill=False,  # Do not fill the rectangles
            weight=1,  # Grid line width
        ).add_to(m)

# Display the map
m

# Save the map as an HTML file using your specified file path
m.save(map_html)

m


NameError: name 'm' is not defined